In [37]:
import json
import numpy as np
import pandas as pd
from binance.client import Client
from binance.enums import *
import math
import datetime
import time
import e # contain os environ that contain binance  keys


#boucle pour se connecter
disconnected = True
while disconnected:
    try :
        client = Client(e.api_key, e.secret_key)
        print("vous etes connecter\n")
        disconnected = False
        connected = True
    except:
        print("impossible de se connecter\nveuillez patientez\n")
#----------------------------------------

#--------process to choose a crypto to rade with----------------------
tickers = client.get_ticker()


list_of_crypto = ['BTC','ETH','XRP','BNB','OCEAN','OGN']
crypto_info = []
for ticker in tickers:
    ticker.pop('weightedAvgPrice')
    ticker.pop('quoteVolume')
    ticker.pop('firstId')
    ticker.pop('lastId')
    ticker.pop('count')
    ticker.pop('highPrice')
    ticker.pop('lowPrice')
    ticker.pop('lastQty')
    ticker.pop('prevClosePrice')
    ticker.pop('bidPrice')
    ticker.pop('bidQty')
    ticker.pop('askPrice')
    ticker.pop('askQty')
    ticker.pop('volume')
    for i in list_of_crypto:
        if (i+'USDT') == ticker['symbol']:
            crypto_info.append(ticker)



impossible de se connecter
veuillez patientez

impossible de se connecter
veuillez patientez

vous etes connecter



In [38]:
crypto_info = pd.DataFrame(crypto_info)

crypto_info

,symbol,priceChange,priceChangePercent,lastPrice,openPrice,openTime,closeTime
0,BTCUSDT,-1423.77000000,-2.521,55052.43000000,56476.20000000,1618809706012,1618896106012
1,ETHUSDT,-104.74000000,-4.687,2130.14000000,2234.88000000,1618809706030,1618896106030
2,BNBUSDT,-21.03780000,-4.031,500.89750000,521.93530000,1618809706026,1618896106026
3,XRPUSDT,-0.19911000,-13.396,1.28719000,1.48630000,1618809706012,1618896106012
4,OGNUSDT,-0.39810000,-20.004,1.59200000,1.99010000,1618809705783,1618896105783
5,OCEANUSDT,-0.14170000,-9.859,1.29560000,1.43730000,1618809700054,1618896100054


In [39]:
#trier les indexes pour que 0 correspondents avec maintenant
crypto_info.sort_values('priceChangePercent',ascending=False,inplace=True)

#refaire les index
crypto_info.reset_index(inplace = True)

#supprimer un colonnes pas important
crypto_info.drop(columns=['index'],inplace=True)



crypto_info



,symbol,priceChange,priceChangePercent,lastPrice,openPrice,openTime,closeTime
0,OCEANUSDT,-0.14170000,-9.859,1.29560000,1.43730000,1618809700054,1618896100054
1,ETHUSDT,-104.74000000,-4.687,2130.14000000,2234.88000000,1618809706030,1618896106030
2,BNBUSDT,-21.03780000,-4.031,500.89750000,521.93530000,1618809706026,1618896106026
3,OGNUSDT,-0.39810000,-20.004,1.59200000,1.99010000,1618809705783,1618896105783
4,BTCUSDT,-1423.77000000,-2.521,55052.43000000,56476.20000000,1618809706012,1618896106012
5,XRPUSDT,-0.19911000,-13.396,1.28719000,1.48630000,1618809706012,1618896106012


In [40]:
#coin that we gonna use to trade
coin_to_trade = crypto_info.iloc[0]['symbol']
#price change percent
price_change = str(crypto_info.iloc[0]['priceChangePercent'])+'%'
print('crypto that we gonna use is {}, price change= {}'.format(coin_to_trade,price_change))
#---------------------------------------------------------------------------------------------


crypto that we gonna use is OCEANUSDT, price change= -9.859%


In [41]:
#boucle pour recuperer le prix
coin_ticker = client.get_symbol_ticker(symbol=coin_to_trade)
coin_price = float(coin_ticker['price'])

print('------------------------------------')
print('prix recuperer')
print(coin_ticker)
coin_price

------------------------------------
prix recuperer
{'symbol': 'OCEANUSDT', 'price': '1.29560000'}


1.2956

In [42]:
#-------pou recupererles klines en 15 min et calculer les SMA-----------------------

# fetch 15 minute klines for the last day up until now
#klines_1min = client.get_historical_klines(coin_to_trade, Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")
klines_15min = client.get_historical_klines(coin_to_trade, Client.KLINE_INTERVAL_15MINUTE, "1 day ago UTC")

print('klines recuperer')


klines recuperer


In [43]:
#changer timestamp en date
for kline in klines_15min:
    kline[0] = datetime.datetime.fromtimestamp(kline[0] / 1e3)

klines = pd.DataFrame(klines_15min)#changer en dataframe
klines.drop(columns=[1,2,3,5,6,7,8,9,10,11],inplace= True)#supprimer les collonnes qui ne sont pas necessaires
klines.columns = ['open_time','close_price']#renommer les colonnes

#creer un SMA15
klines['SMA_15'] = klines.iloc[:,1].rolling(window=15).mean()

#creer un SMA50
klines['SMA_50'] = klines.iloc[:,1].rolling(window=50).mean()

#trier les indexes pour que 0 correspondents avec maintenant
klines.sort_index(ascending=False,inplace=True)

#refaire les index
klines.reset_index(inplace = True)

#supprimer un colonnes pas important
klines.drop(columns=['index'],inplace=True)

klines

,open_time,close_price,SMA_15,SMA_50
0,2021-04-20 07:15:00,1.29560000,1.303113,1.360736
1,2021-04-20 07:00:00,1.30020000,1.309220,1.362772
2,2021-04-20 06:45:00,1.31200000,1.315547,1.365780
3,2021-04-20 06:30:00,1.30140000,1.321253,1.368988
4,2021-04-20 03:45:00,1.26750000,1.327573,1.372534
...,...,...,...,...
81,2021-04-19 08:30:00,1.44620000,NaN,NaN
82,2021-04-19 08:15:00,1.44430000,NaN,NaN
83,2021-04-19 08:00:00,1.44300000,NaN,NaN
84,2021-04-19 07:45:00,1.43470000,NaN,NaN
